In [1]:
import tensorflow as tf

In [2]:
v = tf.Variable(0.0)

In [3]:
(v + 1).numpy()

1.0

In [4]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [5]:
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [6]:
v.assign_add(1)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=6.0>

In [7]:
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [8]:
#求解微分并跟踪
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

In [9]:
grad = t.gradient(loss, w)

In [10]:
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [11]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

In [12]:
dloss_dw = t.gradient(loss, w) 

In [13]:
dloss_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [14]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y

In [15]:
dy_dw = t.gradient(y, w) 

In [16]:
dy_dw

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [17]:
dz_dw = t.gradient(z, w) 

In [18]:
dz_dw

<tf.Tensor: shape=(), dtype=float32, numpy=108.0>

In [19]:
(train_image, train_labels), (test_image, test_labels) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 17s 1us/step


In [20]:
train_image = tf.expand_dims(train_image, -1)

In [21]:
test_image = tf.expand_dims(test_image, -1)

In [22]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [23]:
train_image = tf.cast(train_image / 255, tf.float32)

In [24]:
test_image = tf.cast(test_image / 255, tf.float32)

In [25]:
train_labels = tf.cast(train_labels, tf.int64)

In [26]:
test_labels = tf.cast(test_labels, tf.int64)

In [27]:
dataset = tf.data.Dataset.from_tensor_slices((train_image, train_labels))

In [28]:
test_dataset = tf.data.Dataset.from_tensor_slices((test_image, test_labels))

In [29]:
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [30]:
dataset = dataset.shuffle(10000).batch(32)

In [31]:
test_dataset = test_dataset.batch(32)

In [32]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape = (None, None, 1)),
    tf.keras.layers.Conv2D(32, [3, 3], activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [33]:
#model的可训练参数
model.trainable_variables

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 1, 16) dtype=float32, numpy=
 array([[[[ 1.32877037e-01,  1.87556460e-01, -1.24352708e-01,
            3.85888815e-02, -4.21582013e-02, -1.65927440e-01,
           -1.87321603e-02, -1.00766391e-01,  1.59790367e-02,
            1.69163197e-02, -1.57553345e-01, -1.25417471e-01,
           -2.65746862e-02,  1.92104116e-01,  1.03223786e-01,
            1.80624858e-01]],
 
         [[-7.23953098e-02,  1.68898061e-01,  1.59280583e-01,
            1.44520000e-01,  1.43795863e-01,  3.86726856e-02,
           -1.71341032e-01, -1.41529173e-01,  6.15416914e-02,
            1.24566928e-01, -3.11055183e-02,  4.00313139e-02,
            1.65294960e-01,  7.47096092e-02,  1.58628985e-01,
            7.11383969e-02]],
 
         [[ 5.56776673e-02, -1.44803420e-01, -2.80339271e-02,
           -1.88772082e-01, -4.81897593e-03,  1.43317178e-01,
            1.10292986e-01, -1.80366680e-01,  1.87844783e-02,
            7.61889070e-02,  9.81536657e-02,  7.18925893

In [34]:
optimizer = tf.keras.optimizers.Adam()

In [35]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
features, labels = next(iter(dataset))

In [37]:
features.shape

TensorShape([32, 28, 28, 1])

In [38]:
labels.shape

TensorShape([32])

In [39]:
predictions = model(features)

In [40]:
predictions.shape

TensorShape([32, 10])

In [41]:
tf.argmax(predictions, axis = 1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6], dtype=int64)>

In [42]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([5, 2, 2, 9, 6, 4, 2, 9, 7, 4, 5, 0, 1, 5, 6, 5, 6, 2, 6, 0, 1, 6,
       3, 5, 4, 9, 2, 1, 7, 9, 9, 8], dtype=int64)>

In [43]:
def loss(model, x, y):
    #预测
    y_ = model(x)
    #计算返回loss损失值
    return loss_function(y, y_)

In [44]:
train_loss = tf.keras.metrics.Mean('train_loss')
train_acc = tf.keras.metrics.SparseCategoricalAccuracy('train_acc')

In [45]:
test_loss = tf.keras.metrics.Mean('test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy('test_acc')

In [46]:
def train_step(images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_function(labels, pred)
        #loss_step = loss(model, images, labels)
    #损失与模型可训练参数的梯度
    grads = t.gradient(loss_step, model.trainable_variables)
    #应用优化函数
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_acc(labels, pred)

In [47]:
def test_step(images, labels):
    pred = model(images)
    loss_step = loss_function(labels, pred)
    test_loss(loss_step)
    test_acc(labels, pred)

In [48]:
def train():
    for epoch in range(10):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(images, labels)
        print('Epoch{} is finished, loss = {}, acc = {}'.format(epoch, 
                                                                train_loss.result(),
                                                                train_acc.result())) 
        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(images, labels)                                                        
        print('                     test_loss = {}, test_acc = {}'.format(
                                                     test_loss.result(),
                                                     test_acc.result()))
        train_loss.reset_states()
        train_acc.reset_states()
        test_loss.reset_states()
        test_acc.reset_states()

In [49]:
train()

Epoch0 is finished, loss = 1.7122447490692139, acc = 0.3732999861240387
                     test_loss = 1.3978724479675293, test_acc = 0.5249999761581421
Epoch1 is finished, loss = 1.2994446754455566, acc = 0.5620333552360535
                     test_loss = 1.1335800886154175, test_acc = 0.6312000155448914
Epoch2 is finished, loss = 1.0641064643859863, acc = 0.6588166952133179
                     test_loss = 0.9239758253097534, test_acc = 0.7089999914169312
Epoch3 is finished, loss = 0.8961315751075745, acc = 0.7181166410446167
                     test_loss = 0.8396769165992737, test_acc = 0.7178999781608582
Epoch4 is finished, loss = 0.7885327339172363, acc = 0.7516166567802429
                     test_loss = 0.7127339839935303, test_acc = 0.7824000120162964
Epoch5 is finished, loss = 0.7094447016716003, acc = 0.7794333100318909
                     test_loss = 0.6425023078918457, test_acc = 0.7975000143051147
Epoch6 is finished, loss = 0.6455734372138977, acc = 0.800700008869171

# tf.keras.metrics 汇总计算模块

In [50]:
m = tf.keras.metrics.Mean('acc')

In [51]:
m(10)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [52]:
m(20)

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [53]:
m.result().numpy()

15.0

In [54]:
m([30, 40])

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [55]:
m.result().numpy()

25.0

In [56]:
#重置状态
m.reset_states()

In [57]:
m(1)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [58]:
m(2)

<tf.Tensor: shape=(), dtype=float32, numpy=1.5>

In [59]:
m.result().numpy()

1.5

In [60]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')

In [61]:
a(labels, model(features))

<tf.Tensor: shape=(), dtype=float32, numpy=0.71875>